In [3]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point
from shapely.ops import nearest_points

# --------------------------------------------------
# 0. USER SETTINGS
# --------------------------------------------------

# GPS trajectory to be map-matched
GPS_CSV = "Dec6_Kotturpuram_naviconly_12.53 pm trip.xls"  # <-- change if needed

# DGPS-based network (created earlier)
# EDGES_SHP = "dgps_edges.shp"
# NODES_SHP = "dgps_nodes.shp"

EDGES_SHP = "base_map_edges.shp"
NODES_SHP = "base_map_nodes.shp"

# Output
OUTPUT_CSV = "Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.csv"

# Coordinate system for metric operations (pick correct UTM zone)
utm_crs = "EPSG:32644"   # adjust if your area is in a different UTM zone

# HMM parameters
SEARCH_RADIUS = 50    # meters, candidate search radius
SIGMA = 20            # std dev of GPS noise (for emission)
BETA = 100            # (not used deeply but kept for completeness)

# --------------------------------------------------
# 1. LOAD GPS DATA
# --------------------------------------------------
print("1. Loading GPS data...")

df = pd.read_excel(GPS_CSV)

# assumes 'Date' and 'Time' columns exist; handle multiple formats robustly
if 'Date' in df.columns and 'Time' in df.columns:
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')
    if df['datetime'].isnull().all():
        df['datetime'] = pd.to_datetime(df['Date'], errors='coerce')
elif 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
else:
    raise ValueError("GPS CSV must contain 'Date'+'Time' or 'datetime' columns")

df = df.sort_values("datetime").reset_index(drop=True)

gps_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]),
    crs="EPSG:4326",
)

print("   GPS points:", len(gps_gdf))

# --------------------------------------------------
# 2. LOAD DGPS-BASED NETWORK
# --------------------------------------------------
print("2. Loading DGPS network...")

edges_gdf = gpd.read_file(EDGES_SHP)
nodes_gdf = gpd.read_file(NODES_SHP)

print("   Edges:", len(edges_gdf))
print("   Nodes:", len(nodes_gdf))

# Reproject to UTM for metric distances
gps_gdf_utm = gps_gdf.to_crs(utm_crs)
edges_gdf_utm = edges_gdf.to_crs(utm_crs)

# --------------------------------------------------
# 3. HMM EMISSION PROBABILITY
# --------------------------------------------------
def emission_prob(dist_meters, sigma=20):
    """
    P(observation | state).
    Gaussian w.r.t distance to road segment.
    """
    return np.exp(-0.5 * (dist_meters / sigma) ** 2)


# --------------------------------------------------
# 4. BUILD TRELLIS GRAPH (CANDIDATES + TRANSITIONS)
# --------------------------------------------------
print("3. Building HMM trellis graph...")

trellis = nx.DiGraph()
candidate_map = {}  # time_step -> list of node_ids

# A. Create candidate states for each time step
for t, row in gps_gdf_utm.iterrows():
    gps_point = row.geometry

    # candidate segments within SEARCH_RADIUS
    idxs = edges_gdf_utm.sindex.query(gps_point.buffer(SEARCH_RADIUS))
    cand_segments = edges_gdf_utm.iloc[idxs]

    if cand_segments.empty:
        continue  # no candidates at this time step

    for cand_idx, segment in cand_segments.iterrows():
        dist = gps_point.distance(segment.geometry)
        e_prob = emission_prob(dist, SIGMA)

        node_id = (t, str(cand_idx))  # unique per (time, segment)
        weight = -np.log(e_prob + 1e-9)  # negative log prob

        trellis.add_node(
            node_id,
            weight=weight,
            geometry=segment.geometry,  # UTM linestring
            segment_id=int(cand_idx),   # row index in edges_gdf
        )

        candidate_map.setdefault(t, []).append(node_id)

print("   Time steps with candidates:", len(candidate_map))

# B. Create transitions between consecutive time steps
print("   Linking time steps...")

sorted_times = sorted(candidate_map.keys())

for i in range(len(sorted_times) - 1):
    t_curr = sorted_times[i]
    t_next = sorted_times[i + 1]

    curr_nodes = candidate_map[t_curr]
    next_nodes = candidate_map[t_next]

    for u_node in curr_nodes:
        for v_node in next_nodes:
            u_seg_id = int(trellis.nodes[u_node]["segment_id"])
            v_seg_id = int(trellis.nodes[v_node]["segment_id"])

            # Get logical connectivity from DGPS network (from_id / to_id)
            u_end_node = edges_gdf.loc[u_seg_id, "to_id"]
            v_start_node = edges_gdf.loc[v_seg_id, "from_id"]

            # 1) High probability if edges are connected or same edge
            connected = (u_end_node == v_start_node) or (u_seg_id == v_seg_id)

            if connected:
                trans_prob = 0.9
            else:
                # 2) Otherwise, allow small gaps if end of one is close to start of other
                u_geom = edges_gdf_utm.loc[u_seg_id, "geometry"]
                v_geom = edges_gdf_utm.loc[v_seg_id, "geometry"]

                u_pt = Point(list(u_geom.coords)[-1])
                v_pt = Point(list(v_geom.coords)[0])
                gap = u_pt.distance(v_pt)  # meters in UTM CRS

                if gap < 50:  # allow small jumps at intersections / noise
                    trans_prob = 0.1
                else:
                    trans_prob = 1e-6  # almost impossible

            trans_cost = -np.log(trans_prob)
            next_emission_cost = trellis.nodes[v_node]["weight"]
            total_weight = trans_cost + next_emission_cost

            trellis.add_edge(u_node, v_node, weight=total_weight)

# --------------------------------------------------
# 5. VITERBI (SHORTEST PATH IN TRELLIS)
# --------------------------------------------------
print("4. Solving for optimal path...")

if not candidate_map:
    raise RuntimeError("No candidates found for any time step – check SEARCH_RADIUS / CRS.")

trellis.add_node("START", weight=0)
trellis.add_node("END", weight=0)

first_t = sorted_times[0]
last_t = sorted_times[-1]

for node in candidate_map[first_t]:
    trellis.add_edge("START", node, weight=trellis.nodes[node]["weight"])

for node in candidate_map[last_t]:
    trellis.add_edge(node, "END", weight=0)

try:
    path = nx.shortest_path(trellis, "START", "END", weight="weight")
    path = path[1:-1]  # drop START and END
    print("   Solution found! Path length (states):", len(path))
except nx.NetworkXNoPath:
    print("   Critical Failure: No valid path through trellis.")
    path = []

# --------------------------------------------------
# 6. EXTRACT MATCHED POINTS
# --------------------------------------------------
print("5. Extracting matched points...")

matched_data = []

for node_id in path:
    t, _ = node_id
    seg_id = trellis.nodes[node_id]["segment_id"]
    seg_geom = trellis.nodes[node_id]["geometry"]  # UTM

    orig_point = gps_gdf_utm.iloc[t].geometry
    matched_pt = nearest_points(orig_point, seg_geom)[1]

    matched_data.append(
        {
            "original_index": t,
            "matched_segment_id": int(seg_id),
            "matched_geometry_utm": matched_pt,
        }
    )

matched_df = pd.DataFrame(matched_data)
matched_gdf = gpd.GeoDataFrame(
    matched_df,
    geometry="matched_geometry_utm",
    crs=utm_crs,
)

matched_gdf_wgs = matched_gdf.to_crs("EPSG:4326")

# --------------------------------------------------
# 7. MERGE BACK TO ORIGINAL GPS AND SAVE
# --------------------------------------------------
print("6. Writing output...")

gps_gdf["hmm_matched_lat"] = np.nan
gps_gdf["hmm_matched_lon"] = np.nan

geom_col = matched_gdf_wgs.geometry.name

for _, row in matched_gdf_wgs.iterrows():
    orig_idx = row["original_index"]
    point_geom = row[geom_col]
    gps_gdf.at[orig_idx, "hmm_matched_lat"] = point_geom.y
    gps_gdf.at[orig_idx, "hmm_matched_lon"] = point_geom.x

gps_gdf.drop(columns="geometry").to_csv(OUTPUT_CSV, index=False)
print(f"Done. Saved to {OUTPUT_CSV}")


1. Loading GPS data...
   GPS points: 115
2. Loading DGPS network...
   Edges: 76
   Nodes: 77
3. Building HMM trellis graph...
   Time steps with candidates: 110
   Linking time steps...
4. Solving for optimal path...
   Solution found! Path length (states): 110
5. Extracting matched points...
6. Writing output...
Done. Saved to Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.csv
